In [1]:
# the code is partially based on https://github.com/Zinoex/bound_propagation/blob/main/examples/fashion_mnist.py 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt
from tqdm import tqdm

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = True
device = torch.device("mps" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)

transform = transforms.Compose([
        transforms.PILToTensor(),
        transforms.ConvertImageDtype(torch.float),
        transforms.Lambda(torch.flatten)
    ])


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [25]:
from bound_propagation import BoundModelFactory, HyperRectangle

# set up the network
class Network(nn.Sequential):
    def __init__(self):
        in_size = 28*28
        classes = 10

        super().__init__(
            nn.Linear(in_size, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, classes)
        )

net = Network().to(device)

factory = BoundModelFactory()
net = factory.build(net)

In [3]:
# train the network on clean data
def clean_train(model, num_epochs):
    learning_rate = 0.0001

    opt = optim.Adam(params=net.parameters(), lr=learning_rate)

    ce_loss = torch.nn.CrossEntropyLoss()

    tot_steps = 0

    for epoch in range(1,num_epochs+1):
        t1 = time.time()

        for _, (x_batch, y_batch) in enumerate(train_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            tot_steps += 1
            opt.zero_grad()
            out = model(x_batch)
            batch_loss = ce_loss(out, y_batch)
            batch_loss.backward()
            opt.step()
    
        tot_test, tot_acc = 0.0, 0.0
        for _, (x_batch, y_batch) in enumerate(test_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            out = model(x_batch)
            pred = torch.max(out, dim=1)[1]
            acc = pred.eq(y_batch).sum().item()
            tot_acc += acc
            tot_test += x_batch.size()[0]
        t2 = time.time()

        print('Epoch %d: Accuracy %.5lf [%.2lf seconds]' % (epoch, tot_acc/tot_test, t2-t1))

In [60]:
clean_train(net, 20)
torch.save(net.state_dict(), 'weights_clean.pt')

Epoch 1: Accuracy 0.86100 [8.93 seconds]
Epoch 2: Accuracy 0.89270 [6.33 seconds]
Epoch 3: Accuracy 0.90480 [6.19 seconds]
Epoch 4: Accuracy 0.91400 [6.14 seconds]
Epoch 5: Accuracy 0.92110 [6.11 seconds]
Epoch 6: Accuracy 0.92520 [6.11 seconds]
Epoch 7: Accuracy 0.92970 [6.11 seconds]
Epoch 8: Accuracy 0.93560 [6.09 seconds]
Epoch 9: Accuracy 0.93610 [6.21 seconds]
Epoch 10: Accuracy 0.94140 [6.30 seconds]
Epoch 11: Accuracy 0.94390 [6.04 seconds]
Epoch 12: Accuracy 0.94560 [6.15 seconds]
Epoch 13: Accuracy 0.94680 [6.11 seconds]
Epoch 14: Accuracy 0.94980 [6.22 seconds]
Epoch 15: Accuracy 0.95190 [6.31 seconds]
Epoch 16: Accuracy 0.95280 [6.27 seconds]
Epoch 17: Accuracy 0.95320 [6.11 seconds]
Epoch 18: Accuracy 0.95410 [6.19 seconds]
Epoch 19: Accuracy 0.95530 [6.13 seconds]
Epoch 20: Accuracy 0.95810 [6.12 seconds]


### Interval Analysis

In [14]:
# inputs: images, model, perturbation sizes
# return: the robustness of the inputs
def ibp_analysis(train_loader, net, eps):
    with tqdm(train_loader, unit="batch") as tepoch:
        running_correct = 0
        num_samples = 0
        for i, (x, target) in enumerate(tepoch):
            x, target = x.to(device), target.to(device)
            input_bounds = HyperRectangle.from_eps(x, eps)
            ibp_bounds = net.ibp(input_bounds)
            lower, upper = ibp_bounds.lower, ibp_bounds.upper

            # expand the classes with the same size of bounds [batch_size, 10]
            classes = torch.arange(10, device=device).unsqueeze(0).expand(len(target), -1)
            # get mask with 1 of the correct class and 0 rest of them
            mask = (classes == target.unsqueeze(-1)).to(dtype=lower.dtype)

            upper_sel = (upper * (1-mask)).max(dim=-1)[0] # get the higest upper bounds for wrong classes
            lower_sel = lower[classes == target.unsqueeze(-1)] 

            running_correct += (lower_sel > upper_sel).cpu().float().sum().item()
            num_samples += len(x)
    
    print(f"Robustness: {running_correct / num_samples} with eps {eps}")

In [31]:
eps =np.linspace(0.01, 0.1, 10)
net.load_state_dict(torch.load('weights_clean.pt'))
test_clean(net, test_loader)
for ep in eps:
    ibp_analysis(test_loader, net, ep)


[TEST]
Accuracy: 0.958


100%|██████████| 157/157 [00:02<00:00, 73.40batch/s]


Robustness: 0.0 with eps 0.01


100%|██████████| 157/157 [00:02<00:00, 75.24batch/s]


Robustness: 0.0 with eps 0.020000000000000004


...

Robustness: 0.0 with eps 0.08


100%|██████████| 157/157 [00:01<00:00, 78.72batch/s]


Robustness: 0.0 with eps 0.09000000000000001


100%|██████████| 157/157 [00:01<00:00, 78.83batch/s]

Robustness: 0.0 with eps 0.1


### Robust Training

In [43]:
def adversarial_logit(y_hat, y):
    batch_size = y.size(0)
    classes = torch.arange(10, device=y.device).unsqueeze(0).expand(batch_size, -1)
    mask = (classes == y.unsqueeze(-1)).to(dtype=y_hat.lower.dtype)

    # Take upper bound for logit of all but the correct class where you take the lower bound
    adversarial_logit = (1 - mask) * y_hat.upper + mask * y_hat.lower

    return adversarial_logit

def robust_training(train_loader, net, eps, num_epoch):
    criterion = torch.nn.CrossEntropyLoss().to(device)
    k = 1.0
    for _ in range(num_epoch):
        running_loss = 0.0
        running_cross_entropy = 0.0
        with tqdm(train_loader, unit="batch") as tepoch:
            running_correct = 0
            num_samples = 0
            learning_rate = 5e-4

            opt = optim.Adam(params=net.parameters(), lr=learning_rate)

            # ce_loss = torch.nn.CrossEntropyLoss()
            for i, (x, target) in enumerate(tepoch):
                x, target = x.to(device), target.to(device)

                opt.zero_grad()

                y_hat = net(x)
                cross_entropy = criterion(y_hat, target)


                input_bounds = HyperRectangle.from_eps(x, eps)
                ibp_bounds = net.ibp(input_bounds)

                logit = adversarial_logit(ibp_bounds, target)

                loss = k * cross_entropy + (1 - k) * criterion(logit, target)

                # lower, upper = ibp_bounds.lower, ibp_bounds.upper

                # # expand the classes with the same size of bounds [batch_size, 10]
                # classes = torch.arange(10, device=device).unsqueeze(0).expand(len(target), -1)
                # # get mask with 1 of the correct class and 0 rest of them
                # mask = (classes == target.unsqueeze(-1)).to(dtype=lower.dtype)

                # upper_sel = (upper * (1-mask)).max(dim=-1)[0] # get the higest upper bounds for wrong classes
                # lower_sel = lower[classes == target.unsqueeze(-1)] 

                # loss = (upper_sel - lower_sel).mean()

                loss.backward()
                opt.step()

                # running_correct += (lower_sel > upper_sel).cpu().float().sum().item()
                # num_samples += len(x)
                running_loss += loss.item()
                running_cross_entropy += cross_entropy.item()

                tepoch.set_postfix({'loss_all':running_loss / (i+1), 'loss_clean': running_cross_entropy / (i+1)})
        
        k = max(k - 0.1, 0.5)
        # ibp_analysis(train_loader, net, eps)


    # print(f"Robustness: {running_correct / num_samples} with eps {eps}")

In [44]:
def test_clean(net, test_loader):
    # print('')
    # print('[TEST]')

    correct = 0
    num_samples = 0
    for i, (X, y) in enumerate(test_loader):
        X, y = X.to(device), y.to(device)

        y_hat = net(X)

        predicted = torch.argmax(y_hat, 1)
        correct += (predicted == y).sum().item()

        num_samples += len(y)

    print(f'Accuracy: {correct / num_samples :.3f}')

In [45]:
eps =np.linspace(0.01, 0.1, 10)
# net.load_state_dict(torch.load('weights_clean.pt'))
for ep in eps:
    net = Network().to(device)

    factory = BoundModelFactory()
    net = factory.build(net)

    robust_training(train_loader, net, ep, 20)

    # testing on robustness
    print('robustness:')
    ibp_analysis(test_loader, net, ep)

    # testing on accuracy
    print('accuracy:')
    test_clean(net, test_loader)

100%|██████████| 938/938 [00:20<00:00, 44.88batch/s, loss_all=0.128, loss_clean=0.0737]


robustness:


100%|██████████| 157/157 [00:02<00:00, 78.31batch/s]


Robustness: 0.9223 with eps 0.01
accuracy:
Accuracy: 0.972


100%|██████████| 938/938 [00:20<00:00, 44.91batch/s, loss_all=0.187, loss_clean=0.0973]


...

Robustness: 0.6855 with eps 0.09000000000000001
accuracy:
Accuracy: 0.886


100%|██████████| 938/938 [00:21<00:00, 44.35batch/s, loss_all=0.833, loss_clean=0.451]


robustness:


100%|██████████| 157/157 [00:02<00:00, 77.69batch/s]


Robustness: 0.6775 with eps 0.1
accuracy:
Accuracy: 0.874
